In [4]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
from io import StringIO
import requests
import unicodedata

def remove_special_characters(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

def download_csv_and_show_table():
    source_url = source_entry.get()
    col_index = col_index_entry.get()

    csv_stream = download_csv(source_url)

    if csv_stream:
        df = pd.read_csv(csv_stream)
        if col_index:
            col_index = int(col_index)
            if col_index < len(df.columns):
                df = df.iloc[:, [col_index]]
            else:
                print("Índice de coluna fora do intervalo")
                return
        else:
            df = df.iloc[:, :4]

        for col in df.columns:
            df[col] = df[col].apply(lambda x: remove_special_characters(str(x).upper()))

        # Crie uma janela para exibir a tabela
        data_window = tk.Toplevel(root)
        data_window.title("Dados CSV")
        data_tree = ttk.Treeview(data_window)
        data_tree["columns"] = df.columns
        data_tree.heading("#0", text="Índice")
        for col_id, col_name in enumerate(df.columns):
            data_tree.heading(col_id, text=col_name)
            data_tree.column(col_id, width=100)
        for index, row in df.iterrows():
            data_tree.insert("", "end", text=index, values=[row[i] for i in range(len(df.columns))])
        data_tree.pack()

def download_csv(url):
    response = requests.get(url)
    if response.status_code == 200:
        return StringIO(response.text)
    else:
        return None

# Configuração da janela principal
root = tk.Tk()
root.title("Aplicativo CSV")

# Criação de widgets
source_label = ttk.Label(root, text="URL do CSV:")
source_entry = ttk.Entry(root, width=40)
col_index_label = ttk.Label(root, text="Índice da Coluna (opcional):")
col_index_entry = ttk.Entry(root, width=5)
fetch_button = ttk.Button(root, text="Carregar e Mostrar Tabela", command=download_csv_and_show_table)

# Posicionamento dos widgets
source_label.grid(row=0, column=0, sticky="w")
source_entry.grid(row=0, column=1, columnspan=2, sticky="w")
col_index_label.grid(row=1, column=0, sticky="w")
col_index_entry.grid(row=1, column=1, sticky="w")
fetch_button.grid(row=1, column=2)

# Inicia a interface gráfica
root.mainloop()